In [1]:
using CSV, DataFrames, Dates, LinearAlgebra, Statistics, Gadfly, GLM, Optim, MLBase, LIBSVM, EvalMetrics, NamedArrays, DataTables, MLDataPattern, Tables, Plots, PyCall 

[ Info: Precompiling Gadfly [c91e804a-d5a3-530f-b6f0-dfbca275c004]


In [2]:
using NBInclude
@nbinclude("Functions.ipynb")

PCA_principal_components (generic function with 1 method)

In [30]:
result_folder = "Résultats/5.3/5.3 - "

"Résultats/5.3/5.3 - "

In [3]:
close_pixels = ["42018", "43018", "44018", "42017", "43017", "44017", "42016", "43016","44016", "41015", "42015", "43015", "44015", "41014", "42014", "43014"]
sites = ["4380-01D", "4400-02D", "4420-04D", "4795-01D"]

4-element Vector{String}:
 "4380-01D"
 "4400-02D"
 "4420-04D"
 "4795-01D"

In [4]:
df_rain = CSV.read(raw"Données\close_pixels_24h_accumulation.csv", DataFrame)

filter!(row -> (Dates.month.(row.Date) .>= 5) .& (Dates.month.(row.Date) .<= 10), df_rain)
describe(df_rain)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Any,Any,Int64,DataType
1,Date,,2020-05-01T00:00:00,2021-01-30T11:55:00,2021-10-31T23:50:00,0,DateTime
2,42018_d10min,0.140244,0.0,0.0,133.2,0,Float64
3,42018_d20min,0.280505,0.0,0.0,151.0,0,Float64
4,42018_d30min,0.420771,0.0,0.0,284.2,0,Float64
5,42018_d40min,0.560975,0.0,0.0,302.0,0,Float64
6,42018_d50min,0.701125,0.0,0.0,366.8,0,Float64
7,42018_d1h,0.841318,0.0,0.0,366.8,0,Float64
8,42018_d2h,1.68013,0.0,0.0,366.8,0,Float64
9,42018_d3h,2.51757,0.0,0.0,366.8,0,Float64


In [30]:
df_cso = CSV.read(raw"Données\4380-01D_CSO_10_min_synchro.csv", DataFrame)
rename!(df_cso, ["Date", "CSO_4380-01D"])
df_cso[!, Symbol("Past_CSO_4380-01D")] = vcat(0, [df_cso[i+1,"CSO_4380-01D"] for i in 1:size(df_cso)[1]-1])
for site in ["4400-02D", "4420-04D", "4795-01D"]
    df_site = CSV.read(raw"Données\\"*site*"_CSO_10_min_synchro.csv", DataFrame)
    rename!(df_site, ["Date", "CSO_"*site])
    df_site[!, Symbol("Past_" *site)] = vcat(0, [df_site[i+1,"CSO_"*site] for i in 1:size(df_site)[1]-1])
    df_cso = innerjoin(df_cso, df_site, on = :Date)
end
df_cso

,Date,CSO_4380-01D,Past_CSO_4380-01D,CSO_4400-02D,Past_4400-02D
,DateTime,Bool,Int64,Bool,Int64
1,2020-01-01T00:00:00,0,0,0,0
2,2020-01-01T00:10:00,0,0,0,0
3,2020-01-01T00:20:00,0,0,0,0
4,2020-01-01T00:30:00,0,0,0,0
5,2020-01-01T00:40:00,0,0,0,0
6,2020-01-01T00:50:00,0,0,0,0
7,2020-01-01T01:00:00,0,0,0,0
8,2020-01-01T01:10:00,0,0,0,0
9,2020-01-01T01:20:00,0,0,0,0


In [31]:
df = innerjoin(df_rain, df_cso, on = :Date)

,Date,42018_d10min,42018_d20min,42018_d30min,42018_d40min,42018_d50min
,DateTime,Float64,Float64,Float64,Float64,Float64
1,2020-05-01T00:00:00,0.0,0.0,0.0,0.0,0.0
2,2020-05-01T00:10:00,0.0,0.0,0.0,0.0,0.0
3,2020-05-01T00:20:00,0.0,0.0,0.0,0.0,0.0
4,2020-05-01T00:30:00,0.0,0.0,0.0,0.0,0.0
5,2020-05-01T00:40:00,0.0,0.0,0.0,0.0,0.0
6,2020-05-01T00:50:00,0.0,0.0,0.0,0.0,0.0
7,2020-05-01T01:00:00,0.0,0.0,0.0,0.0,0.0
8,2020-05-01T01:10:00,0.0,0.0,0.0,0.0,0.0
9,2020-05-01T01:20:00,0.0,0.0,0.0,0.0,0.0


In [32]:
CSV.write(raw"Données\Pixels_proches_accumulation_surversejoint",df)

"Données\\Pixels_proches_accumulation_surversejoint"

In [33]:
println(sum(df[:,"CSO_4380-01D"] .== 1))
println(sum(df[:,"CSO_4400-02D"] .== 1))
println(sum(df[:,"CSO_4420-04D"] .== 1))
println(sum(df[:,"CSO_4795-01D"] .== 1))

104
247
20
423


In [5]:
df = CSV.read(raw"Données\Pixels_proches_accumulation_surversejoint", DataFrame)
train = filter(row -> Dates.year(row.Date) == 2020,df)
test = filter(row -> Dates.year(row.Date) == 2021,df)

Row,Date,42018_d10min,42018_d20min,42018_d30min,42018_d40min,42018_d50min,42018_d1h,42018_d2h,42018_d3h,42018_d4h,42018_d6h,42018_d8h,42018_d12h,42018_d18h,42018_d24h,43018_d10min,43018_d20min,43018_d30min,43018_d40min,43018_d50min,43018_d1h,43018_d2h,43018_d3h,43018_d4h,43018_d6h,43018_d8h,43018_d12h,43018_d18h,43018_d24h,44018_d10min,44018_d20min,44018_d30min,44018_d40min,44018_d50min,44018_d1h,44018_d2h,44018_d3h,44018_d4h,44018_d6h,44018_d8h,44018_d12h,44018_d18h,44018_d24h,42017_d10min,42017_d20min,42017_d30min,42017_d40min,42017_d50min,42017_d1h,42017_d2h,42017_d3h,42017_d4h,42017_d6h,42017_d8h,42017_d12h,42017_d18h,42017_d24h,43017_d10min,43017_d20min,43017_d30min,43017_d40min,43017_d50min,43017_d1h,43017_d2h,43017_d3h,43017_d4h,43017_d6h,43017_d8h,43017_d12h,43017_d18h,43017_d24h,44017_d10min,44017_d20min,44017_d30min,44017_d40min,44017_d50min,44017_d1h,44017_d2h,44017_d3h,44017_d4h,44017_d6h,44017_d8h,44017_d12h,44017_d18h,44017_d24h,42016_d10min,42016_d20min,42016_d30min,42016_d40min,42016_d50min,42016_d1h,42016_d2h,42016_d3h,42016_d4h,42016_d6h,42016_d8h,42016_d12h,42016_d18h,42016_d24h,43016_d10min,⋯
,DateTime,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2021-05-01T00:00:00,1.5,3.6,5.4,6.3,9.0,15.5,31.3,36.9,50.6,58.9,75.3,106.5,170.4,188.0,2.7,5.4,6.7,8.2,10.3,18.9,33.3,40.4,52.6,62.2,76.6,102.0,169.5,187.4,2.7,4.8,6.6,7.5,8.4,17.0,27.5,34.8,43.5,53.2,68.4,91.9,154.5,174.9,1.5,3.3,5.4,7.5,10.7,16.3,32.1,37.9,51.7,60.1,73.3,99.5,165.5,191.2,2.4,4.5,7.2,8.2,9.2,16.7,29.6,35.5,47.4,55.8,68.2,92.4,157.3,177.4,2.7,5.9,7.7,8.9,9.9,17.4,28.0,34.5,45.6,54.7,70.1,94.1,158.8,176.9,2.1,3.6,4.9,6.1,9.3,14.9,27.1,33.0,43.7,52.5,65.0,90.1,148.8,170.4,2.4,⋯
2,2021-05-01T00:10:00,1.3,2.8,4.9,6.7,7.6,10.3,31.1,36.7,50.9,59.0,75.7,104.6,171.0,189.3,3.2,5.9,8.6,9.9,11.4,13.5,35.5,40.4,54.8,64.1,78.8,103.4,172.0,190.6,4.2,6.9,9.0,10.8,11.7,12.6,30.7,35.8,47.1,55.9,71.4,94.6,158.0,179.1,2.1,3.6,5.4,7.5,9.6,12.8,32.4,38.2,53.1,61.5,74.4,99.8,167.1,193.3,2.7,5.1,7.2,9.9,10.9,11.9,31.1,36.4,49.2,57.3,70.0,93.8,159.5,180.1,3.2,5.9,9.1,10.9,12.1,13.1,29.7,35.3,47.9,56.7,71.8,95.8,161.4,180.1,2.7,4.8,6.3,7.6,8.8,12.0,28.6,33.6,45.7,53.9,67.0,91.0,151.2,173.1,2.7,⋯
3,2021-05-01T00:20:00,2.1,3.4,4.9,7.0,8.8,9.7,29.0,37.5,52.0,60.4,76.3,104.6,172.6,191.4,2.4,5.6,8.3,11.0,12.3,13.8,34.7,41.6,55.9,65.6,80.6,103.7,173.8,193.0,5.6,9.8,12.5,14.6,16.4,17.3,34.2,40.4,52.1,60.6,76.1,98.4,163.0,184.7,2.4,4.5,6.0,7.8,9.9,12.0,32.1,39.7,54.2,63.2,75.9,99.8,168.9,195.7,3.2,5.9,8.3,10.4,13.1,14.1,31.9,38.7,51.2,59.6,72.3,95.5,162.1,183.3,5.6,8.8,11.5,14.7,16.5,17.7,33.5,39.9,52.6,61.4,76.1,99.0,166.0,185.7,1.5,4.2,6.3,7.8,9.1,10.3,26.9,34.4,46.2,54.4,67.2,90.4,152.1,174.6,2.1,⋯
4,2021-05-01T00:30:00,0.4,2.5,3.8,5.3,7.4,9.2,28.4,37.0,50.6,60.2,73.1,103.2,169.4,191.8,0.6,3.0,6.2,8.9,11.6,12.9,34.1,41.3,55.2,65.2,79.1,102.8,171.2,193.6,0.5,6.1,10.3,13.0,15.1,16.9,33.4,39.7,51.4,60.2,75.6,97.7,160.8,185.2,0.5,2.9,5.0,6.5,8.3,10.4,30.8,39.0,52.0,62.8,75.1,99.0,165.8,196.2,0.5,3.7,6.4,8.8,10.9,13.6,31.2,38.0,50.5,59.5,71.9,94.8,160.2,183.8,0.6,6.2,9.4,12.1,15.3,17.1,32.8,39.3,51.7,61.3,74.6,98.3,163.0,186.3,0.6,2.1,4.8,6.9,8.4,9.7,26.5,33.5,44.4,54.4,66.6,89.5,149.5,175.2,0.6,⋯
5,2021-05-01T00:40:00,0.9,1.3,3.4,4.7,6.2,8.3,27

In [6]:
println(sum(train[:,"CSO_4380-01D"] .== 1))
println(sum(train[:,"CSO_4400-02D"] .== 1))
println(sum(train[:,"CSO_4420-04D"] .== 1))
println(sum(train[:,"CSO_4795-01D"] .== 1))

75
170
16
50


# PCA sans rééchantillonage

In [6]:
for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    
    println(cso_col)
    
    X_train = Matrix(select(train, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
    y_train = train[:,cso_col]
    Z_train = X_train .- mean(X_train, dims= 1)

    X_test = Matrix(select(test, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
    y_test = test[:,cso_col]
    Z_test = X_test .- mean(X_test, dims = 1)

    SVD_train = svd(Z_train)
    var_train = cumulative_variance_plot(SVD_train.S, false)
    
    
    F1_par_composante = DataFrame()
    TP = DataFrame()

    for k in 1:5:200
    
        println(k)
    
        PC_train = PCA_principal_components(SVD_train,k)

        d_proj = DataFrame([Symbol(name) => (Z_train * SVD_train.V[:,1:k])[:,name] for name in 1:k])
        d_proj = hcat(d_proj,DataFrame(CSO = train[:,cso_col]))
        names_train = Symbol.(names(d_proj)[1:end-1])
        
        
        fm_train = glm(Term(:CSO) ~ sum(Term.(Symbol.(names_train))), d_proj, Bernoulli(), LogitLink(), maxiter = 1000)
        
        test_proj = Z_test * SVD_train.V[:, 1:k]
        d_test = DataFrame([Symbol(name) => test_proj[:,name] for name in 1:k])
        d_test = hcat(d_test,DataFrame(CSO = test[:,cso_col]))
    
        scores = convert(Vector{Float64}, MLBase.predict(fm_train, d_proj))

        function fobj(θ::Real)
            scores = convert(Vector{Float64}, MLBase.predict(fm_train, d_proj))
            r = roc(d_proj.CSO, scores, θ)
            return -f1score(r)
        end

        res_vec = optimize(fobj, 0.0, 1.0)
        u = res_vec.minimizer
    

        scores = convert(Vector{Float64}, MLBase.predict(fm_train, d_test))
    
        F1_par_composante = vcat(F1_par_composante, DataFrame(F1 =f1score(roc(d_test.CSO, scores, u)), k = k, Cumulative_var = var_train.Variance[k]))
        TP = vcat(TP, DataFrame(tp = EvalMetrics.true_positive(d_test.CSO, scores, u), k = k, Cumulative_var = var_train.Variance[k]))
    
        CSV.write(result_folder * "PCA_pure_F1_"*cso_col*".csv", F1_par_composante)
        CSV.write(result_folder * "PCA_pure_TP_"*cso_col*".csv", TP)
    end
    
    
end

CSO_4380-01D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4400-02D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4420-04D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4795-01D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196


In [40]:
total = DataFrame()
for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    
    println(cso_col)
    this_site = CSV.read(result_folder * "PCA_pure_F1_"*cso_col*".csv",DataFrame)
    this_site[!, :Site] .= cso_col[5:end]
    
    total = vcat(total, this_site)
    
end
CSV.write(result_folder * "PCA_pure_F1.csv", total)

CSO_4380-01D
CSO_4400-02D
CSO_4420-04D
CSO_4795-01D


"Résultats/5.3/5.3 - PCA_pure_F1.csv"

# PCA avec RUS, PCA avant RUS

In [41]:
F1_par_composante = DataFrame()
TP = DataFrame()

for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    
    println(cso_col)
    
    X_train = Matrix(select(train, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
    y_train = train[:,cso_col]
    Z_train = X_train .- mean(X_train, dims= 1)

    X_test = Matrix(select(test, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
    y_test = test[:,cso_col]
    Z_test = X_test .- mean(X_test, dims = 1)

    SVD_train = svd(Z_train)
    var_train = cumulative_variance_plot(SVD_train.S, false)


    for k in 1:5:200
    
        println(k)
    
    
    
        PC_train = PCA_principal_components(SVD_train,k)

        d_proj = DataFrame([Symbol(name) => (Z_train * SVD_train.V[:,1:k])[:,name] for name in 1:k])
        d_proj = hcat(d_proj,DataFrame(CSO = train[:, cso_col]))
        
        for i_RUS in 1:30
        
            under_PC_projected = DataFrame(DataTable(undersample(row->row.CSO, DataTable(d_proj))))
            names_train = Symbol.(names(d_proj)[1:end-1])
            
            local fm_train
            
            try
                fm_train = glm(Term(:CSO) ~ sum(Term.(Symbol.(names_train))), under_PC_projected, Bernoulli(), LogitLink(), maxiter = 1000)
            catch e
                println(e)
            else
                test_proj = Z_test * SVD_train.V[:, 1:k]
                d_test = DataFrame([Symbol(name) => test_proj[:,name] for name in 1:k])
                d_test = hcat(d_test,DataFrame(CSO = test[:, cso_col]))

                u = .5
                scores = convert(Vector{Float64}, MLBase.predict(fm_train, d_test))

                F1_par_composante = vcat(F1_par_composante, DataFrame(F1 =f1score(roc(d_test.CSO, scores, u)), k = k, Cumulative_var = var_train.Variance[k], RUS = i_RUS))
                TP = vcat(TP, DataFrame(tp = EvalMetrics.true_positive(d_test.CSO, scores, u), k = k, Cumulative_var = var_train.Variance[k], RUS = i_RUS))

                CSV.write(result_folder* "PCA_RUS_F1_"*cso_col*".csv", F1_par_composante)
                CSV.write(result_folder* "PCA_RUS_TP_"*cso_col*".csv", TP)
            end
        end
    
    end
    
    
end

CSO_4380-01D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
PosDefException(87)
PosDefException(87)
PosDefException(87)
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4400-02D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4420-04D
1
6
PosDefException(7)
PosDefException(7)
11
16
21
PosDefException(22)
PosDefException(22)
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4795-01D
1
6
11
16
21
26
31
36
41
46
51
56
PosDefException(57)
61
66
PosDefException(66)
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196


In [43]:
F1s = DataFrame()
TPs = DataFrame()
for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    this_F1 = CSV.read(result_folder* "PCA_RUS_F1_"*cso_col*".csv", DataFrame)
    this_TP = CSV.read(result_folder* "PCA_RUS_TP_"*cso_col*".csv", DataFrame)
    
    this = combine(groupby(this_F1, :k), [:F1, :Cumulative_var] .=> mean)
    rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    F1s = vcat(F1s, this)
    
    this = combine(groupby(this_TP, :k), [:tp, :Cumulative_var] .=> mean)
    rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    TPs = vcat(TPs, this)
    
end
CSV.write(result_folder* "moyenne_PCA_RUS_F1_.csv", F1s)
CSV.write(result_folder* "moyenne_PCA_RUS_TP_.csv", TPs)

"Résultats/5.3/5.3 - moyenne_PCA_RUS_TP_.csv"

# PCA avec ROS, PCA avant ROS

In [7]:
F1_par_composante = DataFrame()
TP = DataFrame()

for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    
    println(cso_col)
    
    X_train = Matrix(select(train, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
    y_train = train[:,cso_col]
    Z_train = X_train .- mean(X_train, dims= 1)

    X_test = Matrix(select(test, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
    y_test = test[:,cso_col]
    Z_test = X_test .- mean(X_test, dims = 1)

    SVD_train = svd(Z_train)
    var_train = cumulative_variance_plot(SVD_train.S, false)

    for k in 1:5:200
    
        println(k)
    
    
    
        PC_train = PCA_principal_components(SVD_train,k)

        d_proj = DataFrame([Symbol(name) => (Z_train * SVD_train.V[:,1:k])[:,name] for name in 1:k])
        d_proj = hcat(d_proj,DataFrame(CSO = train[:, cso_col]))
        
        
        for i_ROS in 1:30
        
            under_PC_projected = DataFrame(DataTable(oversample(row->row.CSO, DataTable(d_proj))))
            names_train = Symbol.(names(d_proj)[1:end-1])
            
            local fm_train
            try
                fm_train = glm(Term(:CSO) ~ sum(Term.(Symbol.(names_train))), under_PC_projected, Bernoulli(), LogitLink(), maxiter = 1000)
                
            catch e
                println(e)
            else
                test_proj = Z_test * SVD_train.V[:, 1:k]
                d_test = DataFrame([Symbol(name) => test_proj[:,name] for name in 1:k])
                d_test = hcat(d_test,DataFrame(CSO = test[:, cso_col]))

                u = .5
                scores = convert(Vector{Float64}, MLBase.predict(fm_train, d_test))

                F1_par_composante = vcat(F1_par_composante, DataFrame(F1 =f1score(roc(d_test.CSO, scores, u)), k = k, Cumulative_var = var_train.Variance[k], ROS = i_ROS))
                TP = vcat(TP, DataFrame(tp = EvalMetrics.true_positive(d_test.CSO, scores, u), k = k, Cumulative_var = var_train.Variance[k], ROS = i_ROS))

                CSV.write(result_folder* "PCA_ROS_F1_"*cso_col*".csv", F1_par_composante)
                CSV.write(result_folder* "PCA_ROS_TP_"*cso_col*".csv", TP)
            end
        end
    
    end
    
    
end

CSO_4380-01D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4400-02D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4420-04D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4795-01D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196


In [22]:
F1s = DataFrame()
TPs = DataFrame()
for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    this_F1 = CSV.read(result_folder* "PCA_ROS_F1_"*cso_col*".csv", DataFrame)
    this_TP = CSV.read(result_folder* "PCA_ROS_TP_"*cso_col*".csv", DataFrame)
    
    this = combine(groupby(this_F1, :k), [:F1, :Cumulative_var] .=> mean)
    rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    F1s = vcat(F1s, this)
    
    this = combine(groupby(this_TP, :k), [:tp, :Cumulative_var] .=> mean)
    rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    TPs = vcat(TPs, this)
    
end
CSV.write(result_folder* "moyenne_PCA_ROS_F1_.csv", F1s)
CSV.write(result_folder* "moyenne_PCA_ROS_TP_.csv", TPs)

"Résultats/5.3/5.3 - moyenne_PCA_ROS_TP_.csv"

# PCA avec SMOTE, PCA avant SMOTE

In [10]:
imb = pyimport("imblearn") 
smt = imb.over_sampling.SMOTE(sampling_strategy = 1.0)

F1_par_composante = DataFrame()
TP = DataFrame()

for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    
    println(cso_col)
    
    X_train = Matrix(select(train, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
    y_train = train[:,cso_col]
    Z_train = X_train .- mean(X_train, dims= 1)

    X_test = Matrix(select(test, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
    y_test = test[:,cso_col]
    Z_test = X_test .- mean(X_test, dims = 1)

    SVD_train = svd(Z_train)
    var_train = cumulative_variance_plot(SVD_train.S, false)


    for k in 1:5:200
    
        println(k)
    
    
    
        PC_train = PCA_principal_components(SVD_train,k)

        d_proj = DataFrame([Symbol(name) => (Z_train * SVD_train.V[:,1:k])[:,name] for name in 1:k])
        d_proj = hcat(d_proj,DataFrame(CSO = train[:, cso_col]))
        
        names_train = Symbol.(names(d_proj)[1:end-1])
        
        for i_SMOTE in 1:30
        
            d_first, first_CSO = smt.fit_resample(Array(select(d_proj, Not([:CSO]))), d_proj.CSO) 
            smote_PC_projected = DataFrame(hcat(d_first, first_CSO), vcat(names_train,Symbol.([:CSO])))
        
            
            local fm_train
            try
                fm_train = glm(Term(:CSO) ~ sum(Term.(Symbol.(names_train))), smote_PC_projected, Bernoulli(), LogitLink(), maxiter = 1000)
            
            catch e
                println(e)
            else
                test_proj = Z_test * SVD_train.V[:, 1:k]
                d_test = DataFrame([Symbol(name) => test_proj[:,name] for name in 1:k])
                d_test = hcat(d_test,DataFrame(CSO = test[:, cso_col]))

                u = .5
                scores = convert(Vector{Float64}, MLBase.predict(fm_train, d_test))

                F1_par_composante = vcat(F1_par_composante, DataFrame(F1 =f1score(roc(d_test.CSO, scores, u)), k = k, Cumulative_var = var_train.Variance[k], SMOTE = i_SMOTE))
                TP = vcat(TP, DataFrame(tp = EvalMetrics.true_positive(d_test.CSO, scores, u), k = k, Cumulative_var = var_train.Variance[k], SMOTE = i_SMOTE))

                CSV.write(result_folder* "PCA_SMOTE_F1_"*cso_col*".csv", F1_par_composante)
                CSV.write(result_folder* "PCA_SMOTE_TP_"*cso_col*".csv", TP)
            end
        end
        
    end
    
    
end

CSO_4380-01D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4400-02D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4420-04D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4795-01D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196


In [23]:
F1s = DataFrame()
TPs = DataFrame()
for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    this_F1 = CSV.read(result_folder* "PCA_SMOTE_F1_"*cso_col*".csv", DataFrame)
    this_TP = CSV.read(result_folder* "PCA_SMOTE_TP_"*cso_col*".csv", DataFrame)
    
    this = combine(groupby(this_F1, :k), [:F1, :Cumulative_var] .=> mean)
    rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    F1s = vcat(F1s, this)
    
    this = combine(groupby(this_TP, :k), [:tp, :Cumulative_var] .=> mean)
    rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    TPs = vcat(TPs, this)
    
end
CSV.write(result_folder* "moyenne_PCA_SMOTE_F1_.csv", F1s)
CSV.write(result_folder* "moyenne_PCA_SMOTE_TP_.csv", TPs)

"Résultats/5.3/5.3 - moyenne_PCA_SMOTE_TP_.csv"

# PCA avec RUS, RUS avant PCA

In [8]:
F1_par_composante = DataFrame()
TP = DataFrame()
preds = DataFrame()
preds = vcat(preds, DataFrame())

for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    
    println(cso_col)
    
    for i_RUS in 1:30
    
        under_df = DataFrame(DataTable(undersample(row->row[Symbol(cso_col)], DataTable(train))))

        X_under = Matrix(select(under_df, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
        Z_under = X_under .- mean(X_under, dims = 1)
        SVD_under = svd(Z_under)
        var_under = cumulative_variance_plot(SVD_under.S, false)

        X_test = Matrix(select(test, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
        y_test = test[:,cso_col]
        Z_test = X_test .- mean(X_test, dims = 1)
    

        for k_under in 1:5:size(var_under)[1]
    
            println(k_under)
    
            PC_under = PCA_principal_components(SVD_under, k_under)
            d_under = DataFrame([Symbol(name) => (Z_under * SVD_under.V[:,1:k_under])[:, name] for name in 1:k_under])
            d_under = hcat(d_under,DataFrame(CSO = under_df[:,cso_col]))
            names_train = Symbol.(names(d_under)[1:end-1])
            
            local fm_train
            try

                fm_train = glm(Term(:CSO) ~ sum(Term.(Symbol.(names_train))), d_under, Bernoulli(), LogitLink(), maxiter = 1000)
            catch e
                println(e)
            else
                
                test_proj = Z_test * SVD_under.V[:, 1:k_under]
                d_test = DataFrame([Symbol(name) => test_proj[:,name] for name in 1:k_under])
                d_test = hcat(d_test,DataFrame(CSO = test[:, cso_col]))

                u = .5
                scores = convert(Vector{Float64}, MLBase.predict(fm_train, d_test))

                F1_par_composante = vcat(F1_par_composante, DataFrame(F1 =f1score(roc(d_test.CSO, scores, u)), k = k_under,Cumulative_var = var_under.Variance[k_under], RUS = i_RUS))
                TP = vcat(TP, DataFrame(tp = EvalMetrics.true_positive(d_test.CSO, scores, u), k = k_under,Cumulative_var = var_under.Variance[k_under], RUS = i_RUS))

                CSV.write(result_folder * "RUS_PCA_F1_"*cso_col*".csv", F1_par_composante)
                CSV.write(result_folder * "RUS_PCA_TP_"*cso_col*".csv", TP)
            end
    
        end
    end
    
end

CSO_4380-01D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
1


LoadError: InterruptException:

In [70]:
F1s = DataFrame()
TPs = DataFrame()
for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    this_F1 = CSV.read(result_folder* "RUS_PCA_F1_"*cso_col*".csv", DataFrame)
    this_TP = CSV.read(result_folder* "RUS_PCA_TP_"*cso_col*".csv", DataFrame)
    
    this = combine(groupby(this_F1, :k), [:F1, :Cumulative_var] .=> mean)
    rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    F1s = vcat(F1s, this)
    
    this = combine(groupby(this_TP, :k), [:tp, :Cumulative_var] .=> mean)
    rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    TPs = vcat(TPs, this)
    
end
CSV.write(result_folder* "moyenne_RUS_PCA_F1_.csv", F1s)
CSV.write(result_folder* "moyenne_RUS_PCA_TP_.csv", TPs)

"Résultats/5.3/5.3 - moyenne_RUS_PCA_TP_.csv"

# PCA avec ROS, ROS avant PCA

In [30]:
F1_par_composante = DataFrame()
TP = DataFrame()

for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
 
    
    println(cso_col)
    
    for i_ROS in 1:30
    
        under_df = DataFrame(DataTable(oversample(row->row[Symbol(cso_col)], DataTable(train))))

        X_under = Matrix(select(under_df, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
        Z_under = X_under .- mean(X_under, dims = 1)
        SVD_under = svd(Z_under)
        var_under = cumulative_variance_plot(SVD_under.S, false)

        X_test = Matrix(select(test, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
        y_test = test[:,cso_col]
        Z_test = X_test .- mean(X_test, dims = 1)


        for k_under in 1:5:min(size(var_under)[1],200)

            println(k_under)

            PC_under = PCA_principal_components(SVD_under, k_under)
            d_under = DataFrame([Symbol(name) => (Z_under * SVD_under.V[:,1:k_under])[:, name] for name in 1:k_under])
            d_under = hcat(d_under,DataFrame(CSO = under_df[:,cso_col]))
            names_train = Symbol.(names(d_under)[1:end-1])

            local fm_train
            try
                fm_train = glm(Term(:CSO) ~ sum(Term.(Symbol.(names_train))), d_under, Bernoulli(), LogitLink(), maxiter = 1000)
            catch e
                println(e)
            else
                test_proj = Z_test * SVD_under.V[:, 1:k_under]
                d_test = DataFrame([Symbol(name) => test_proj[:,name] for name in 1:k_under])
                d_test = hcat(d_test,DataFrame(CSO = test[:, cso_col]))

                u = .5
                scores = convert(Vector{Float64}, MLBase.predict(fm_train, d_test))

                F1_par_composante = vcat(F1_par_composante, DataFrame(F1 =f1score(roc(d_test.CSO, scores, u)), k = k_under,Cumulative_var = var_under.Variance[k_under], ROS = i_ROS))
                TP = vcat(TP, DataFrame(tp = EvalMetrics.true_positive(d_test.CSO, scores, u), k = k_under,Cumulative_var = var_under.Variance[k_under], ROS = i_ROS))

                CSV.write(result_folder * "ROS_PCA_F1_"*cso_col*".csv", F1_par_composante)
                CSV.write(result_folder * "ROS_PCA_TP_"*cso_col*".csv", TP)
            end

        end
    end
    
end

CSO_4380-01D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4400-02D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
1
6
11
1

In [31]:
F1s = DataFrame()
TPs = DataFrame()
for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    this_F1 = CSV.read(result_folder* "ROS_PCA_F1_"*cso_col*".csv", DataFrame)
    this_TP = CSV.read(result_folder* "ROS_PCA_TP_"*cso_col*".csv", DataFrame)
    
    this = combine(groupby(this_F1, :k), [:F1, :Cumulative_var] .=> mean)
    rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    F1s = vcat(F1s, this)
    
    this = combine(groupby(this_TP, :k), [:tp, :Cumulative_var] .=> mean)
    rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    TPs = vcat(TPs, this)
    
end
CSV.write(result_folder* "moyenne_ROS_PCA_F1_.csv", F1s)
CSV.write(result_folder* "moyenne_ROS_PCA_TP_.csv", TPs)

"Résultats/5.3/5.3 - moyenne_ROS_PCA_TP_.csv"

# PCA avec SMOTE, SMOTE avant PCA

In [34]:
F1_par_composante = DataFrame()
TP = DataFrame()

for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    
    println(cso_col)
    
    for i_SMOTE in 1:30
    
        d_first, first_CSO = smt.fit_resample(Array(train[:,2:end-4]), train[:,cso_col]) 
        smote_df = DataFrame(hcat(d_first, first_CSO), vcat(names(select(train, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]))),cso_col))

        under_df = smote_df

        X_under = Matrix(select(under_df, Not(cso_col)))
        Z_under = X_under .- mean(X_under, dims = 1)
        SVD_under = svd(Z_under)
        var_under = cumulative_variance_plot(SVD_under.S, false)

        X_test = Matrix(select(test, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
        y_test = test[:,cso_col]
        Z_test = X_test .- mean(X_test, dims = 1)


        F1_par_composante = DataFrame()
        TP = DataFrame()

        for k_under in 1:5:min(size(var_under)[1],200)

            println(k_under)

            PC_under = PCA_principal_components(SVD_under, k_under)
            d_under = DataFrame([Symbol(name) => (Z_under * SVD_under.V[:,1:k_under])[:, name] for name in 1:k_under])
            d_under = hcat(d_under,DataFrame(CSO = under_df[:,cso_col]))
            names_train = Symbol.(names(d_under)[1:end-1])
            
            local fm_train
            try
                fm_train = glm(Term(:CSO) ~ sum(Term.(Symbol.(names_train))), d_under, Bernoulli(), LogitLink(), maxiter = 1000)
            catch e
                println(e)
            else

                test_proj = Z_test * SVD_under.V[:, 1:k_under]
                d_test = DataFrame([Symbol(name) => test_proj[:,name] for name in 1:k_under])
                d_test = hcat(d_test,DataFrame(CSO = test[:, cso_col]))

                u = .5
                scores = convert(Vector{Float64}, MLBase.predict(fm_train, d_test))

                F1_par_composante = vcat(F1_par_composante, DataFrame(F1 =f1score(roc(d_test.CSO, scores, u)), k = k_under, Cumulative_var = var_under.Variance[k_under], SMOTE = i_SMOTE))
                TP = vcat(TP, DataFrame(tp = EvalMetrics.true_positive(d_test.CSO, scores, u), k = k_under,Cumulative_var = var_under.Variance[k_under], SMOTE = i_SMOTE))

                CSV.write(result_folder * "SMOTE_PCA_F1_"*cso_col*".csv", F1_par_composante)
                CSV.write(result_folder * "SMOTE_PCA_TP_"*cso_col*".csv", TP)
            end

        end
    end 
    
end

CSO_4380-01D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
CSO_4400-02D
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81
86
91
96
101
106
111
116
121
126
131
136
141
146
151
156
161
166
171
176
181
186
191
196
1
6
11
1

In [47]:
F1s = DataFrame()
TPs = DataFrame()
for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    this_F1 = CSV.read(result_folder* "SMOTE_PCA_F1_"*cso_col*".csv", DataFrame)
    this_TP = CSV.read(result_folder* "SMOTE_PCA_TP_"*cso_col*".csv", DataFrame)
    
    this = combine(groupby(this_F1, :k), [:F1, :Cumulative_var] .=> mean)
    rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    F1s = vcat(F1s, this)
    
    this = combine(groupby(this_TP, :k), [:tp, :Cumulative_var] .=> mean)
    rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    TPs = vcat(TPs, this)
    
end
CSV.write(result_folder* "moyenne_SMOTE_PCA_F1_.csv", F1s)
CSV.write(result_folder* "moyenne_SMOTE_PCA_TP_.csv", TPs)

"Résultats/5.3/5.3 - moyenne_SMOTE_PCA_TP_.csv"

# PLS DA

In [32]:
using RCall
reval("library(mdatools)")

ncomps = vcat([1], 2:2:200)
comp_number = length(ncomps)

for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    preds = 0
    println(cso_col)
    
    X_train = Matrix(select(train, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
    y_train = train[:,cso_col]

    X_test = Matrix(select(test, Not(["Date", "CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"])))
    y_test = test[:,cso_col]
    
    @rput X_train 
    @rput X_test
    @rput y_train
    @rput y_test
    
    reval("mod = plsda(
      X_train,
      y_train,
      ncomp = min(nrow(X_train) - 1, ncol(X_train), 200),
      center = TRUE,
      scale = FALSE,
      cv = NULL,
      x.test = X_test,
      c.test = y_test,
      ncomp.selcrit = 'wold',
      classname = TRUE)")
    
    preds = rcopy(R"predict(mod, X_test, y_test)")
    
    tns = preds[:tn][ncomps]
    tps = preds[:tp][ncomps]
    fns = preds[:fn][ncomps]
    fps = preds[:fp][ncomps]
    
    F1s = [tps[i] ./ (tps[i] .+ 0.5*(fps[i] .+ fns[i]) ) for i in 1:length(tns)]
    
    CSV.write(result_folder * cso_col*"_PLS-DA_F1_multiple_ncomp.csv", DataFrame(Components = ncomps, F1 = F1s))
    CSV.write(result_folder * cso_col*"_PLS-DA_TP_multiple_ncomp.csv", DataFrame(Components = ncomps, TP = tps))
    CSV.write(result_folder * cso_col*"_PLS-DA_TN_multiple_ncomp.csv", DataFrame(Components = ncomps, TN = tns))
    CSV.write(result_folder * cso_col*"_PLS-DA_FP_multiple_ncomp.csv", DataFrame(Components = ncomps, FP = fps))
    CSV.write(result_folder * cso_col*"_PLS-DA_FN_multiple_ncomp.csv", DataFrame(Components = ncomps, FN = fns))
    
    
end

CSO_4380-01D
CSO_4400-02D
CSO_4420-04D
CSO_4795-01D


In [48]:
F1s = DataFrame()
for cso_col in ["CSO_4380-01D", "CSO_4400-02D", "CSO_4420-04D", "CSO_4795-01D"]
    this_F1 = CSV.read(result_folder * cso_col*"_PLS-DA_F1_multiple_ncomp.csv", DataFrame)
    rename!(this_F1, :Components => :k)
    
    this = combine(groupby(this_F1, :k), :F1 .=> mean)
    #rename!(this, :Cumulative_var_mean => :Cumulative_var)
    this[!, :Site] .= cso_col[5:end]
    this[!,:Cumulative_var] .= 0
    F1s = vcat(F1s, this)
    
end
CSV.write(result_folder* "moyenne_PLSDA_F1_.csv", F1s)


"Résultats/5.3/5.3 - moyenne_PLSDA_F1_.csv"